## 1. A skeleton class structure for documents

In [1]:
import numpy as np
import codecs
import nltk
import re
import math 
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer
from itertools import repeat


class Corpus():
    
    """ 
    The Corpus class represents a document collection
     
    """
    def __init__(self, doc_data, stopword_file, clean_length):
        """
        Notice that the __init__ method is invoked everytime an object of the class
        is instantiated
        """
        

        #Initialise documents by invoking the appropriate class
        self.docs = [Document(doc[0], doc[1], doc[2]) for doc in doc_data] 
        
        self.N = len(self.docs)
        self.clean_length = clean_length
        
        #get a list of stopwords
        self.create_stopwords(stopword_file, clean_length)
        
        #stopword removal, token cleaning and stemming to docs
        self.clean_docs(2)
        
        #create vocabulary
        self.corpus_tokens()
        
    def clean_docs(self, length):
        """ 
        Applies stopword removal, token cleaning and stemming to docs
        """
        for doc in self.docs:
            doc.token_clean(length)
            doc.stopword_remove(self.stopwords)
            doc.stem()        
    
    def create_stopwords(self, stopword_file, length):
        """
        description: parses a file of stowords, removes words of length 'length' and 
        stems it
        input: length: cutoff length for words
               stopword_file: stopwords file to parse
        """
        
        with codecs.open(stopword_file,'r','utf-8') as f: raw = f.read()
        
        self.stopwords = (np.array([PorterStemmer().stem(word) 
                                    for word in list(raw.splitlines()) if len(word) > length]))
        
     
    def corpus_tokens(self):
        """
        description: create a set of all all tokens or in other words a vocabulary
        """
        
        #initialise an empty set
        self.token_set = set()
        for doc in self.docs:
            self.token_set = self.token_set.union(doc.tokens) 
            
    def document_term_matrix(self):
        """
        description:  returns a D by V array of frequency counts
        """  
        # subroutine: computes the counts of each vocabulary in the document
        def counts(doc):
            # initialize a matrix
            term_mat = [0]*len(self.token_set)
            for token in doc.tokens:
                term_mat[list(self.token_set).index(token)] = term_mat[list(self.token_set).index(token)] + 1
            return term_mat;
            
        self.doc_term_matrix = []
        
        for doc in self.docs:
            self.doc_term_matrix.append([doc.pres + " " + doc.year, counts(doc)])


      
    def tf_idf(self):
        """
        description:  returns a D by V array of tf-idf scores
        """
        # Compute inverse document frequency 
        idf = [0]*len(self.token_set)
        for token in self.token_set:
            ind = 0
            for doc in self.docs:
                if token in doc.tokens:
                    ind += 1 
            idf[list(self.token_set).index(token)] = math.log(self.N/ind)
        
        # Create a subroutine that computes tf_idf for one document
        def tfidf(doc):
            term_mat = [0]*len(self.token_set)
            for token in doc.tokens:
                term_mat[list(self.token_set).index(token)] = term_mat[list(self.token_set).index(token)] + 1 
        
            for i,term in enumerate(term_mat):
                if term != 0:
                    term_mat[i] = (1 + math.log(term)) * idf[i]
            return term_mat;
        
        #tf_idf
        self.tf_idf_matrix = []
        for doc in self.docs:
            self.tf_idf_matrix.append([doc.pres + " " + doc.year, tfidf(doc)])
            
            
        
    def dict_rank(self, n, dictionary, token_repr):
        """
        description:  returns the top n documents based on a given dictionary and represenation of tokens
        """
        if token_repr == "tf-idf":
            self.tf_idf()
            representation = self.tf_idf_matrix
            
        if token_repr == "doc-term":
            self.document_term_matrix()
            representation = self.doc_term_matrix
            
        # Return top n docs based on dictionary given
        score = []
        x=self.token_set
        x=list(x)
        for token in x: 
            try:
                score.append(dictionary[token])
            except: 
                score.append(0)

        # get a vector with all the scores in order
        score=[int(x) for x in score]
        rank = {}
        elements=range(len(representation))
   
        for i in elements:
            rank[representation[i][0]] = np.dot(score,representation[i][1])
            
        # Get sorted view of the keys.
        s = sorted(rank, key=rank.get, reverse=True)[0:(n-1)]
        
        ranking = {}
        for key in s:
            ranking[key] =  rank[key]
        
        return ranking 

In [2]:
class Document():
    
    """ The Doc class rpresents a class of individul documents
    
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
        
        
    def token_clean(self,length):

        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """

        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])


    def stopword_remove(self, stopwords):

        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """

        
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])


    def stem(self):

        """
        description: Stem tokens with Porter Stemmer.
        """
        
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])





Load the document and create the corpus


In [3]:
def parse_text(textraw, regex):
    """takes raw string and performs two operations
    1. Breaks text into a list of speech, president and speech
    2. breaks speech into paragraphs
    """
    prs_yr_spch_reg = re.compile(regex, re.MULTILINE|re.DOTALL)
    
    #Each tuple contains the year, last ane of the president and the speech text
    prs_yr_spch = prs_yr_spch_reg.findall(textraw)
    
    #convert immutabe tuple to mutable list
    prs_yr_spch = [list(tup) for tup in prs_yr_spch]
    
    for i in range(len(prs_yr_spch)):
        prs_yr_spch[i][2] = prs_yr_spch[i][2].replace('\n', '')
    
    #sort
    prs_yr_spch.sort()
    
    return(prs_yr_spch)

In [4]:
text = open("/Users/ainalopez/Downloads/text_mining-master/data/pres_speech/sou_all copy.txt", 'r').read()
# text = open("/home/yaroslav/Projects/text_mining/data/pres_speech/sou_all.txt", 'r').read()
regex = "_(\d{4}).*?_[a-zA-Z]+.*?_[a-zA-Z]+.*?_([a-zA-Z]+)_\*+(\\n{2}.*?)\\n{3}"
pres_speech_list = parse_text(text, regex)

#Instantite the corpus class
# corpus = Corpus(pres_speech_list, '/home/yaroslav/Projects/text_mining/data/stopwords/stopwords.txt', 2)
corpus = Corpus(pres_speech_list, '/Users/ainalopez/Downloads/text_mining-master-2/data/stopwords/stopwords.txt', 2)

## 2. tf_idf matrix and SVD

In [5]:
# Compute tf_idf matrix 
corpus.tf_idf()
matrix = corpus.tf_idf_matrix

X =[] 
presidents = []

for row in matrix:
    X.append(row[1])  #= #np.append(X, row[1], axis=0) 
    presidents.append(row[0])

X = np.array(X)


In [54]:
# Compute svd 
sing_values_nb = 200
 
U, s, V = np.linalg.svd(X)
X_hat = np.dot(U[:,0:(sing_values_nb-1)] * s[0:(sing_values_nb-1)], V[0:(sing_values_nb-1),:])

236
236


# 3. Cosine Similarity of documents

In [89]:
def cosine_similarity(doc1, doc2):
    return np.dot(doc1, doc2) / ( math.sqrt(np.dot(doc1, doc1))* math.sqrt(np.dot(doc2, doc2)) )

similarity_X = np.zeros((len(X), len(X)))
similarity_X_hat = np.zeros((len(X_hat), len(X_hat)))

for i in range(len(X)):
    for j in range(len(X)):
        similarity_X[i][j] = cosine_similarity(X[i], X[j])
        
for i in range(len(X_hat)):
    for j in range(len(X_hat)):
        similarity_X_hat[i][j] = cosine_similarity(X_hat[i], X_hat[j])
  

In [91]:
# Add labels per president 



[  1.00000000e+00   7.49326794e-01   7.51415662e-01   7.69472364e-01
   7.90452706e-01   4.34195597e-01   7.80456758e-01   4.89702452e-01
   5.48881075e-01   4.79137936e-01   6.71030648e-01   6.36541279e-01
   6.13853717e-01   5.27436790e-01   6.30542818e-01   5.91960182e-01
   4.72158937e-01   4.70753323e-01   6.68473408e-01   7.20662281e-01
   6.25009698e-01   5.41988987e-01   5.69118430e-01   3.64309390e-01
   3.67186655e-01   3.27858307e-01   6.17343439e-01   6.67380166e-01
   5.01802466e-01   1.34750177e-01   3.27550343e-01   3.71668487e-01
   1.81623111e-01   3.20056234e-01   1.61827790e-01   3.12842859e-01
   3.06853137e-01   2.09709611e-01   1.27639730e-01   1.95065406e-01
   2.07997270e-01   3.26363602e-01   2.50856805e-01   3.35286591e-01
   2.24071340e-01   3.17540865e-01   3.51993151e-01   3.90887320e-01
   3.25197211e-01   3.29905384e-01   3.73251469e-01   2.75478603e-01
   2.18282261e-01   1.92925939e-01   2.76055469e-01   2.57769437e-01
   2.28380882e-01   2.39454558e-01